## Snap Amazon Data Preprocessing

In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np 
from sklearn.preprocessing import LabelEncoder

##### Extracting Data from Snap

In [1]:
#!wget -o Dataset/amazon-meta.txt https://snap.stanford.edu/data/bigdata/amazon/amazon-meta.txt.gz

In [2]:
#!wget -o Dataset/amazon0601.txt 

In [2]:
#Looking at data sample
!head -25 Dataset/amazon-meta.txt

# Full information about Amazon Share the Love products
Total items: 548552

Id:   0
ASIN: 0771044445
  discontinued product

Id:   1
ASIN: 0827229534
  title: Patterns of Preaching: A Sermon Sampler
  group: Book
  salesrank: 396585
  similar: 5  0804215715  156101074X  0687023955  0687074231  082721619X
  categories: 2
   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Preaching[12368]
   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Sermons[12370]
  reviews: total: 2  downloaded: 2  avg rating: 5
    2000-7-28  cutomer: A2JW67OY8U6HHK  rating: 5  votes:  10  helpful:   9
    2003-12-14  cutomer: A2VE83MZF98ITY  rating: 5  votes:   6  helpful:   5

Id:   2
ASIN: 0738700797
  title: Candlemas: Feast of Flames
  group: Book
  salesrank: 168596


#### Data Conversion and Preprocessing

In [3]:
from Helpers import convert_data

In [4]:
collection = convert_data("./Dataset/amazon-meta.txt")

In [5]:
df = pd.DataFrame(data=collection[2:])
df.head(5)

,Id,ASIN,title,group,salesrank,no_sim,similar,categories,catlists,reviews,revdicts
0,1,0827229534,Patterns of Preaching: A Sermon Sampler,Book,396585,5,"[0804215715, 156101074X, 0687023955, 068707423...",2,"[[Books[283155], Subjects[1000], Religion & Sp...","{'total': 2.0, 'downloaded': 2.0, 'avg rating'...","[{'date': '2000-7-28', 'customer': 'A2JW67OY8U..."
1,2,0738700797,Candlemas: Feast of Flames,Book,168596,5,"[0738700827, 1567184960, 1567182836, 073870052...",2,"[[Books[283155], Subjects[1000], Religion & Sp...","{'total': 12.0, 'downloaded': 12.0, 'avg ratin...","[{'date': '2001-12-16', 'customer': 'A11NCO6YT..."
2,3,0486287785,World War II Allied Fighter Planes Trading Cards,Book,1270652,0,[],1,"[[Books[283155], Subjects[1000], Home & Garden...","{'total': 1.0, 'downloaded': 1.0, 'avg rating'...","[{'date': '2003-7-10', 'customer': 'A3IDGASRQA..."
3,4,0842328327,Life Application Bible Commentary: 1 and 2 Tim...,Book,631289,5,"[0842328130, 0830818138, 0842330313, 084232861...",5,"[[Books[283155], Subjects[1000], Religion & Sp...","{'total': 1.0, 'downloaded': 1.0, 'avg rating'...","[{'date': '2004-8-19', 'customer': 'A2591BUPXC..."
4,5,1577943082,Prayers That Avail Much for Business: Executive,Book,455160,5,"[157794349X, 0892749504, 1577941829, 089274956...",2,"[[Books[283155], Subjects[1000], Religion & Sp...","{'total': 0.0, 'downloaded': 0.0, 'avg rating'...",[]


In [6]:
##Decomposing Columns which contain a complex structure within them

In [7]:
pd.DataFrame(df.catlists.explode())["catlists"].apply(lambda x: len(str(x).split(","))).max()

### Data Transformation

In [ ]:
normrev = pd.json_normalize(df['reviews']) 
#normalizing data using json
df_main = df[['Id','ASIN','title','group','salesrank','no_sim','categories']]
df_main = df_main.join(normrev)
df_main["salesrank"] = df_main["salesrank"].fillna(0).replace("",0)
df_main["no_sim"] = df_main["no_sim"].fillna(0).replace("",0)
df_main["categories"] = df_main["categories"].fillna(0).replace("",0)
df_main["total"] = df_main["total"].fillna(0)
df_main["downloaded"] = df_main["downloaded"].fillna(0)
df_main["salesrank"] = df_main["salesrank"].astype(int)
df_main["no_sim"] = df_main["no_sim"].astype(int)
df_main["categories"] = df_main["categories"].astype(int)
df_main['total'] = df_main['total'].astype(int)
df_main['downloaded'] = df_main['downloaded'].astype(int)
df_main.columns = ["Id","ASIN","Title","Group","Salesrank","NbrOfSimilar","NbrOfCategories","TotalReviews","Downloaded","AvgRating"]
df_main.head(5)

,Id,ASIN,Title,Group,Salesrank,NbrOfSimilar,NbrOfCategories,TotalReviews,Downloaded,AvgRating
0,1,0827229534,Patterns of Preaching: A Sermon Sampler,Book,396585,5,2,2,2,5.0
1,2,0738700797,Candlemas: Feast of Flames,Book,168596,5,2,12,12,4.5
2,3,0486287785,World War II Allied Fighter Planes Trading Cards,Book,1270652,0,1,1,1,5.0
3,4,0842328327,Life Application Bible Commentary: 1 and 2 Tim...,Book,631289,5,5,1,1,4.0
4,5,1577943082,Prayers That Avail Much for Business: Executive,Book,455160,5,2,0,0,0.0


In [ ]:
df_main.to_csv('Dataset/main.csv',index=False)

### Product Review Data

In [ ]:
df_reviews = df[['Id','ASIN','revdicts']]
rev_explode = pd.DataFrame(df_reviews.apply(lambda row: row['revdicts'],axis=1).explode(),columns=["reviews"])
df_reviews = df_reviews.join(rev_explode)
df_rev = df_reviews.join(pd.json_normalize(df_reviews["reviews"]))
df_rev = df_rev[['Id','ASIN','date','customer','rating','votes','helpful']]
df_rev["votes"] = df_rev["votes"].fillna(0)
df_rev["helpful"] = df_rev["helpful"].fillna(0)
df_rev["votes"] = df_rev["votes"].astype(int)
df_rev["helpful"] = df_rev["helpful"].astype(int)
df_rev['date'] = pd.to_datetime(df_rev['date'], infer_datetime_format=True)
df_rev.columns = ["Id","ASIN","Date","Customer","Rating","Votes","Helpful"]
df_rev.head(5)

/var/folders/_l/nyk3mgld6gl9lvm8gzrjd2pm0000gn/T/ipykernel_54050/2516623846.py:10: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_rev['date'] = pd.to_datetime(df_rev['date'], infer_datetime_format=True)


,Id,ASIN,Date,Customer,Rating,Votes,Helpful
0,1,0827229534,2000-07-28,A2JW67OY8U6HHK,5.0,10,9
0,1,0827229534,2000-07-28,A2JW67OY8U6HHK,5.0,10,9
1,2,0738700797,2003-12-14,A2VE83MZF98ITY,5.0,6,5
1,2,0738700797,2003-12-14,A2VE83MZF98ITY,5.0,6,5
1,2,0738700797,2003-12-14,A2VE83MZF98ITY,5.0,6,5


In [ ]:
df_rev.to_csv('Dataset/reviews.csv',index=False)

### Product Category Data

In [ ]:
df_categories = df[['Id','ASIN','catlists']]
df_cat = pd.DataFrame(df_categories.apply(lambda row: row["catlists"],axis=1).explode(),columns=["levels"])
df_cat["levels2"] = df_cat.apply(lambda row: str(row["levels"])[1:-1],axis=1)
df_cat["L1"] = df_cat.apply(lambda row: row["levels2"].split(",")[0].replace("'","") if len(row["levels2"].split(","))>0 else "",axis=1)
df_cat["L2"] = df_cat.apply(lambda row: row["levels2"].split(",")[1].replace("'","") if len(row["levels2"].split(","))>1 else "",axis=1)
df_cat["L3"] = df_cat.apply(lambda row: row["levels2"].split(",")[2].replace("'","") if len(row["levels2"].split(","))>2 else "",axis=1)
df_cat["L4"] = df_cat.apply(lambda row: row["levels2"].split(",")[3].replace("'","") if len(row["levels2"].split(","))>3 else "",axis=1)
# for i in range(13):
#     df_cat[f"L{i+1}"] = df_cat.apply(lambda row: row["levels2"].split(",")[i].replace("'","") if len(row["levels2"].split(","))>i else np.nan,axis=1)
df_cat.drop(["levels","levels2"],axis=1,inplace=True)
df_cat = df_categories.join(df_cat)
# df_cat.drop("catlists",axis=1,inplace=True)
df_cat = df_cat[['Id','ASIN','L1','L2','L3','L4']]
df_cat.head(5)

,Id,ASIN,L1,L2,L3,L4
0,1,0827229534,Books[283155],Subjects[1000],Religion & Spirituality[22],Christianity[12290]
0,1,0827229534,Books[283155],Subjects[1000],Religion & Spirituality[22],Christianity[12290]
1,2,0738700797,Books[283155],Subjects[1000],Religion & Spirituality[22],Earth-Based Religions[12472]
1,2,0738700797,Books[283155],Subjects[1000],Religion & Spirituality[22],Earth-Based Religions[12472]
2,3,0486287785,Books[283155],Subjects[1000],Home & Garden[48],Crafts & Hobbies[5126]


In [ ]:
df_cat.to_csv("Dataset/categories.csv",index=False)

### Similar Products Data

In [ ]:
df_similar = df[['Id','ASIN','similar']]
sim_explode = df_similar.apply(lambda row:
    str(row['similar']).replace("[","").replace("]","").split(","), axis=1).explode()
df_sim = df_similar.join(pd.DataFrame(sim_explode,columns=['Similar']))
df_sim.drop("similar",axis=1,inplace=True)
df_sim['Similar'] = df_sim['Similar'].apply(lambda x: str(x).replace("'",""))
df_sim.head()

,Id,ASIN,Similar
0,1,0827229534,0804215715
0,1,0827229534,156101074X
0,1,0827229534,0687023955
0,1,0827229534,0687074231
0,1,0827229534,082721619X


In [ ]:
df_sim.to_csv("Dataset/similars.csv",index=False)

##### End of File